TO DO:
1. Fix counties for unclaimed
2. Fix counties for unidentified
3. Re-export state-level json (now that county fields have been updated within the databases and American Samoa has been removed)
4. Export county-level json - 
    * any records with no county get pulled (either into separate geojson, or to separate state FIPS with no county key and nonsense coordinates [south pole])
    * make sure state name included as field with each database, not just FIPS code

In [76]:
# import necessary packages
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import geojson

1. rework state data - also have FIPS
2. finish cleaning county data
3. format county data - Name and county FIPS code
2. see how bad city data would be

Notes:
1. Edits to Missing_04182020.csv prior to import
  * Virgin Islands (to United States Virgin Islands), 
  * Tennesse (to Tennessee), and 
  * Northern Mariana Islands (to Commonwealth of the Northern Mariana Islands)
2. Edits to Unidentified_04182020.csv prior to import
  * Virgin Islands (to United States Virgin Islands)

In [77]:
# Read in csvs
city_df = pd.read_csv('cities.csv')

In [78]:
county_centroids_df = pd.read_csv('county_centroids.csv', encoding='Windows-1252')

In [79]:
state_centroids_df = pd.read_csv('state_centroids.csv')

In [80]:
missing_df = pd.read_csv('Missing_04182020.csv')

In [81]:
unclaimed_df = pd.read_csv('Unclaimed_04182020.csv')

In [82]:
unidentified_df = pd.read_csv('Unidentified_04182020.csv')

In [83]:
# check dataframe
state_centroids_df.head()

,OBJECTID,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
0,1,28,1779790,0400000US28,28,MS,Mississippi,0,1.215340e+11,3.926920e+09,1,-89.665425,32.751473
1,2,37,1027616,0400000US37,37,NC,North Carolina,0,1.259240e+11,1.346607e+10,2,-79.380052,35.542238
2,3,40,1102857,0400000US40,40,OK,Oklahoma,0,1.776630e+11,3.374588e+09,3,-97.508293,35.583486
3,4,51,1779803,0400000US51,51,VA,Virginia,0,1.022580e+11,8.528532e+09,4,-78.883285,37.512967
4,5,54,1779805,0400000US54,54,WV,West Virginia,0,6.226647e+10,4.890285e+08,5,-80.613731,38.642587


# PART 1: Summary data (count for all 3 databases, by state)

### 1/4: Get count of missing person cases

In [ ]:
missing_count = missing_df.groupby('State').count()
len(missing_count)
missing_count.head()

In [ ]:
# Remove all columns except case number count
missing_count = missing_count.drop(columns=['DLC','Last Name', 'First Name', 'Missing Age', 'City', 'County', 'Sex', 'Race / Ethnicity', 'Date Modified'])

In [ ]:
# add column for state (since state is now index)
missing_count['State'] = missing_count.index

In [ ]:
# rename case number count column
missing_count = missing_count.rename(columns = {'Case Number': 'Missing_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
missing_count.head()

In [ ]:
# make dictionary of states and missing person counts
missing_dict = dict(zip(missing_count.State, missing_count.Missing_CaseCount))
len(missing_dict)
# missing_dict

### 2/4: Get count of unclaimed persons

In [ ]:
unclaimed_count = unclaimed_df.groupby('State').count()
# len(unclaimed_count)
unclaimed_count.head()

In [ ]:
# Remove all columns except case number count
unclaimed_count = unclaimed_count.drop(columns=['DBF','Last Name', 'First Name', 'Sex', 'Race / Ethnicity', 'City', 'County', 'Date Modified'])

In [ ]:
# add column for state (since state is now index)
unclaimed_count['State'] = unclaimed_count.index

In [ ]:
# rename case number count column
unclaimed_count = unclaimed_count.rename(columns = {'Case Number': 'Unclaimed_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unclaimed_count.head()

In [ ]:
# make dictionary of states and unclaimed case counts
unclaimed_dict = dict(zip(unclaimed_count.State, unclaimed_count.Unclaimed_CaseCount))
len(unclaimed_dict)
# unclaimed_dict

### 3/4: Get count of unidentified persons

In [ ]:
unidentified_count = unidentified_df.groupby('State').count()
# len(unidentified_count)
unidentified_count.head()

In [ ]:
# Remove all columns except case number count
unidentified_count = unidentified_count.drop(columns=['DBF','Age From', 'Age To', 'City', 'County', 'Sex', 'Race / Ethnicity', 'Date Modified'])

In [ ]:
# add column for state (since state is now index)
unidentified_count['State'] = unidentified_count.index

In [ ]:
# rename case number count column
unidentified_count = unidentified_count.rename(columns = {'Case Number': 'Unidentified_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unidentified_count.head()

In [ ]:
# make dictionary of states and unidentified case counts
unidentified_dict = dict(zip(unidentified_count.State, unidentified_count.Unidentified_CaseCount))
len(unidentified_dict)
# unidentified_dict

### 4/4: Make summary dataframe

In [ ]:
# make a new summary dataframe based on the state centroids
summary_df = state_centroids_df
# sort by state name
summary_df = summary_df.sort_values(by=['NAME'])
summary_df.head()

In [ ]:
len(summary_df)

#### 4a: Add Missing Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the missing_dict, to pull the correct missing case count for each state
summary_df['Missing_Count'] = summary_df['NAME'].map(missing_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
missing_null_df = summary_df.loc[summary_df['Missing_Count'].isnull()]
missing_null_df

#### 4b: Add Unclaimed Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the unclaimed_dict, to pull the correct unclaimed case count for each state
summary_df['Unclaimed_Count'] = summary_df['NAME'].map(unclaimed_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
unclaimed_null_df = summary_df.loc[summary_df['Unclaimed_Count'].isnull()]
unclaimed_null_df

#### 4b: Add Unidentified Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the unidentified_dict, to pull the correct unidentified case count for each state
summary_df['Unidentified_Count'] = summary_df['NAME'].map(unidentified_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
unidentified_null_df = summary_df.loc[summary_df['Unidentified_Count'].isnull()]
unidentified_null_df

#### 4c: Add column for total count of cases in all three databases for each state

In [ ]:
summary_df['Total_Count'] = summary_df.iloc[:, -4:-1].sum(axis=1)
summary_df.head(20)

#### 4c: Convert to geodataframe and export as GeoJSON

In [ ]:
# check final summary_df
summary_df.head()

In [ ]:
# Convert dataframe to geodataframe
summary_gdf = gpd.GeoDataFrame(summary_df, geometry=gpd.points_from_xy(x=summary_df.Lon_dd, y=summary_df.Lat_dd))
summary_gdf.head()

In [ ]:
# write to geoJSON
summary_gdf.to_file("JSON/summary_counts.json", driver="GeoJSON", encoding='utf-8')

# Part 2 - Get state-level GeoJSON with data from all 3 databases

### 2a: Add in State FIPS column to each database dataframe

In [84]:
# make dictionary of states and state FIPS code
state_dict = dict(zip(state_centroids_df.NAME, state_centroids_df.STATEFP))
state_dict
# state_dict['Alaska']
# len(state_dict)

{'Mississippi': 28,
 'North Carolina': 37,
 'Oklahoma': 40,
 'Virginia': 51,
 'West Virginia': 54,
 'Louisiana': 22,
 'Michigan': 26,
 'Massachusetts': 25,
 'Idaho': 16,
 'Florida': 12,
 'Nebraska': 31,
 'Washington': 53,
 'New Mexico': 35,
 'Puerto Rico': 72,
 'South Dakota': 46,
 'Texas': 48,
 'California': 6,
 'Alabama': 1,
 'Georgia': 13,
 'Pennsylvania': 42,
 'Missouri': 29,
 'Colorado': 8,
 'Utah': 49,
 'Tennessee': 47,
 'Wyoming': 56,
 'New York': 36,
 'Kansas': 20,
 'Alaska': 2,
 'Nevada': 32,
 'Illinois': 17,
 'Vermont': 50,
 'Montana': 30,
 'Iowa': 19,
 'South Carolina': 45,
 'New Hampshire': 33,
 'Arizona': 4,
 'District of Columbia': 11,
 'American Samoa': 60,
 'United States Virgin Islands': 78,
 'New Jersey': 34,
 'Maryland': 24,
 'Maine': 23,
 'Hawaii': 15,
 'Delaware': 10,
 'Guam': 66,
 'Commonwealth of the Northern Mariana Islands': 69,
 'Rhode Island': 44,
 'Kentucky': 21,
 'Ohio': 39,
 'Wisconsin': 55,
 'Oregon': 41,
 'North Dakota': 38,
 'Arkansas': 5,
 'Indiana': 1

#### 1/3: add state FIPS codes to missing persons dataframe

In [85]:
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [86]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
missing_df['State_FIPS'] = missing_df['State'].map(state_dict)
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


In [87]:
# Check unique values in new dataframe field
missing_df['State_FIPS'].unique()

array([ 1,  2,  4,  5,  6,  8, 69,  9, 10, 11, 12, 13, 66, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 72, 44, 45, 46, 47, 48, 78, 49, 50, 51,
       53, 54, 55, 56], dtype=int64)

In [88]:
## If any nulls, check to see what is producing null values [NOTE: fixed by making changes to csv fields, as noted where csvs are imported]
# mis_test_df = missing_df.loc[missing_df['State_FIPS'].isnull()]
# mis_test_df.head()

In [89]:
# check missing df
missing_df.head()
# len(missing_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


#### 2/3: add state FIPS codes to unclaimed persons dataframe

In [90]:
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20


In [91]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unclaimed_df['State_FIPS'] = unclaimed_df['State'].map(state_dict)
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20,36.0
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20,36.0
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20,36.0


In [92]:
# Check unique values in new dataframe field
unclaimed_df['State_FIPS'].unique()

array([17., 36., 25.,  6., 53., 15., 32., 16., 48., 49., 47., 11., 33.,
       35., 40., 42., 39., 29.,  5., 13.,  1., 34.,  4., 12., 55.,  9.,
        8., 26., 22., 18., 19., 41., 23., nan, 45., 56., 54., 51.,  2.,
       28., 21., 37., 10., 38.])

In [93]:
## If any nulls, check to see what is producing null values [NOTE: cannot be addressed, as these cases do not have a state or county assigned]
unc_test_df = unclaimed_df.loc[unclaimed_df['State_FIPS'].isnull()]
unc_test_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
3380,UCP5339,12/3/16,Williams,Barbara,Female,Asian,NaN,NaN,NaN,5/3/18,NaN
4774,UCP2489,3/26/15,Lupien,Daryl,Male,NaN,NaN,NaN,NaN,8/18/16,NaN
5023,UCP4001,11/2/14,Brown,David,Male,White / Caucasian,NaN,NaN,NaN,5/11/17,NaN
5883,UCP3962,3/22/10,Gonzalez,David,Male,Hispanic / Latino,NaN,NaN,NaN,5/11/17,NaN
5950,UCP3937,12/7/09,Togovnick,Bernice,Female,White / Caucasian,NaN,NaN,NaN,5/11/17,NaN
6183,UCP3901,1/21/09,Carney,John,Male,White / Caucasian,NaN,NaN,NaN,5/11/17,NaN
6341,UCP3689,6/5/08,Guzman,Ralph,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/17,NaN
6560,UCP3824,7/2/07,Vasquez,Rafael,Male,Hispanic / Latino,NaN,NaN,NaN,5/4/17,NaN
6698,UCP3672,9/22/06,Barajas,Epifanio,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/17,NaN
6719,UCP3668,8/5/06,Huff,Paul,Male,White / Caucasian,NaN,NaN,NaN,5/2/17,NaN


#### 3/3: add state FIPS codes to unidentified persons dataframe

In [94]:
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020


In [95]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unidentified_df['State_FIPS'] = unidentified_df['State'].map(state_dict)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020,36
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020,41
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020,36
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020,36
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020,40


In [96]:
# Check unique values in new dataframe field
unidentified_df['State_FIPS'].unique()

array([36, 41, 40, 49, 47, 48,  9,  4, 17, 32, 53, 24, 45, 34, 26, 30, 12,
       72, 29, 21,  6,  8, 35, 13, 15,  2,  1, 37, 25, 54, 42, 22, 28, 51,
        5, 39, 16, 33, 55, 18, 20, 31, 11, 10, 27, 23, 50, 38, 44, 46, 56,
       19, 66, 78], dtype=int64)

In [97]:
## If any nulls, check to see what is producing null values [NOTE: fixed by making changes to csv fields, as noted where csvs are imported]
# uni_test_df = unidentified_df.loc[unidentified_df['State_FIPS'].isnull()]
# uni_test_df.head()

### Construct GeoJSON with correct structure

Goal format:
{
"type": "Feature",
    "name": "Wisconsin",
    "properties": {
        "missing": [ ],
        "unclaimed": [ ],
        "unidentified": [ ],
        "filtered": [ ]
    }
    "geometry": {
          "type": "Point",
          "coordinates": [
            -117.79750667,
            36.03755926
          ]
}
* each array will be a list of dictionaries. Each dictionary = one case. Keys = headers

In [23]:
# get headers
missing_header = list(missing_df.columns.values)
print("missing header:", missing_header)
unclaimed_header = list(unclaimed_df.columns.values)
print("unclaimed header:", unclaimed_header)
unidentified_header = list(unidentified_df.columns.values)
print("unidentified header:", unidentified_header)

missing header: ['Case Number', 'DLC', 'Last Name', 'First Name', 'Missing Age', 'City', 'County', 'State', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS']
unclaimed header: ['Case Number', 'DBF', 'Last Name', 'First Name', 'Sex', 'Race / Ethnicity', 'City', 'County', 'State', 'Date Modified', 'State_FIPS']
unidentified header: ['Case Number', 'DBF', 'Age From', 'Age To', 'City', 'County', 'State', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS']


In [24]:
# check df
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020,36
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020,41
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020,36
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020,36
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020,40


In [25]:
# check data types
# type(missing_df['Missing Age'][0])

In [26]:
# check centroids
state_centroids_df.head()

,OBJECTID,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
0,1,28,1779790,0400000US28,28,MS,Mississippi,0,1.215340e+11,3.926920e+09,1,-89.665425,32.751473
1,2,37,1027616,0400000US37,37,NC,North Carolina,0,1.259240e+11,1.346607e+10,2,-79.380052,35.542238
2,3,40,1102857,0400000US40,40,OK,Oklahoma,0,1.776630e+11,3.374588e+09,3,-97.508293,35.583486
3,4,51,1779803,0400000US51,51,VA,Virginia,0,1.022580e+11,8.528532e+09,4,-78.883285,37.512967
4,5,54,1779805,0400000US54,54,WV,West Virginia,0,6.226647e+10,4.890285e+08,5,-80.613731,38.642587


In [27]:
# sort each database by statename
# sort missing by state name
state_centroids_df = state_centroids_df.sort_values(by=['NAME'])
state_centroids_df.head()

,OBJECTID,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
17,18,1,1779775,0400000US01,1,AL,Alabama,0,1.311740e+11,4.593327e+09,18,-86.828092,32.790364
27,28,2,1785533,0400000US02,2,AK,Alaska,0,1.478840e+12,2.454820e+11,28,-152.680813,64.530206
37,38,60,1802701,0400000US60,60,AS,American Samoa,0,1.977591e+08,1.307244e+09,38,-170.718268,-14.300454
35,36,4,1779777,0400000US04,4,AZ,Arizona,0,2.941990e+11,1.027338e+09,36,-111.664418,34.293110
52,53,5,68085,0400000US05,5,AR,Arkansas,0,1.347690e+11,2.962860e+09,53,-92.439268,34.899745


In [28]:
# sort each database by statename
# sort missing by state name
missing_df = missing_df.sort_values(by=['State'])
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
708,MP64666,1/16/2020,Edwards,Katrina,16,Mobile,Mobile,Alabama,Female,White / Caucasian,3/4/2020,1
668,MP54784,9/5/2018,Mickles,Ernest,75,Prichard,Mobile,Alabama,Male,Black / African American,11/13/2019,1
667,MP52935,9/27/2018,Bradley,Leon,86,Marion Junction,Dallas,Alabama,Male,Black / African American,11/13/2019,1
666,MP58186,5/22/2019,Robinson,Ramondus,31,Birmingham,Jefferson,Alabama,Male,Black / African American,2/14/2020,1
665,MP2688,6/18/1959,Barter,Daniel,4,Lillian,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1


In [29]:
# sort missing by state name
unclaimed_df = unclaimed_df.sort_values(by=['State'])
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
2516,UCP5127,10/28/17,Elmore,Jeanne,Female,White / Caucasian,Birmingham,Jefferson,Alabama,1/11/19,1.0
5382,UCP2769,5/26/13,Black,Billy,Male,White / Caucasian,Birmingham,Jefferson,Alabama,9/20/18,1.0
2989,UCP4416,4/22/17,Harris,Scott,Male,White / Caucasian,Birmingham,Jefferson,Alabama,1/11/19,1.0
5095,UCP2760,8/7/14,Allison,Larry,Male,White / Caucasian,Birmingham,Jefferson,Alabama,9/20/18,1.0
5406,UCP2770,4/11/13,Williams,Darryl,Male,Black / African American,Birmingham,Jefferson,Alabama,9/20/18,1.0


In [30]:
# sort missing by state name
unidentified_df = unidentified_df.sort_values(by=['State'])
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
11644,UP14998,2/19/1998,60.0,100.0,Bessemer,Jefferson,Alabama,Female,White / Caucasian,3/18/2020,1
9876,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1
3362,UP13959,4/20/2006,NaN,NaN,Tuscaloosa,Tuscaloosa,Alabama,Male,Uncertain,2/13/2020,1
10468,UP13985,3/27/2014,18.0,99.0,Birmingham,Jefferson,Alabama,Female,Uncertain,3/18/2020,1
216,UP58389,5/30/2019,35.0,55.0,Birmingham,Jefferson,Alabama,Male,White / Caucasian,3/18/2020,1


In [31]:
## Create state array
state_array = []
## for each state in state_centroids...
i = 0
while i < len(state_centroids_df):
    state_dict = {}
    state_dict["type"] = "Feature"
    state_dict["name"] = state_centroids_df["NAME"][i]
    state_dict["name_abbr"] = state_centroids_df["STUSPS"][i]
    state_dict["state_FIPS"] = str(state_centroids_df["STATEFP"][i])
    state_dict["properties"] = {}
    missing_array = []
    j = 0
    while j < len(missing_df):
        # check if state_fips matches
        if missing_df['State_FIPS'][j] == state_centroids_df["STATEFP"][i]:
            missing_dict = {}
            for item in missing_header:
                missing_dict[item] = str(missing_df[item][j])
            # append dictionary to missing array
            missing_array.append(missing_dict)
        # increment j
        j += 1
    state_dict["properties"]['missing'] = missing_array     
    
    unclaimed_array = []
    k = 0
    while k < len(unclaimed_df):
        # check if state_fips matches
        if unclaimed_df['State_FIPS'][k] == state_centroids_df["STATEFP"][i]:
            unclaimed_dict = {}
            for item in unclaimed_header:
                unclaimed_dict[item] = str(unclaimed_df[item][k])
            # append dictionary to unclaimed array
            unclaimed_array.append(unclaimed_dict)
        # increment k
        k += 1
    state_dict["properties"]['unclaimed'] = unclaimed_array
    
    unidentified_array = []
    l = 0
    while l < len(unidentified_df):
        # check if state_fips matches
        if unidentified_df['State_FIPS'][l] == state_centroids_df["STATEFP"][i]:
            unidentified_dict = {}
            for item in unidentified_header:
                unidentified_dict[item] = str(unidentified_df[item][l])
            # append dictionary to unclaimed array
            unidentified_array.append(unidentified_dict)
        # increment l
        l += 1
    state_dict["properties"]['unidentified'] = unidentified_array
    
    state_dict["properties"]['filtered'] = []
    ## set geometry
    state_dict["geometry"] = {}
    state_dict["geometry"]["type"] = "Point"
    state_dict["geometry"]["coordinates"] = [state_centroids_df['Lon_dd'][i], state_centroids_df['Lat_dd'][i]]
    ## append state dictionary to array
    state_array.append(state_dict)
    # increment interator
    i += 1
# state_array

In [32]:
# check item in array
state_array[0]['properties']['unclaimed']

[{'Case Number': 'UCP64285',
  'DBF': '2/22/13',
  'Last Name': 'Dillard',
  'First Name': 'Joseph',
  'Sex': 'Male',
  'Race / Ethnicity': 'Black / African American',
  'City': 'Jackson',
  'County': 'Hinds',
  'State': 'Mississippi',
  'Date Modified': '4/8/20',
  'State_FIPS': '28.0'},
 {'Case Number': 'UCP5008',
  'DBF': 'nan',
  'Last Name': 'Knight',
  'First Name': 'Rachel',
  'Sex': 'Female',
  'Race / Ethnicity': 'White / Caucasian',
  'City': 'nan',
  'County': 'Scott',
  'State': 'Mississippi',
  'Date Modified': '1/3/19',
  'State_FIPS': '28.0'}]

In [33]:
# Create FeatureCollection
state_feature_collection = {}
state_feature_collection["type"] = "FeatureCollection"
state_feature_collection["features"] = state_array
# state_feature_collection

In [34]:
# Convert FeatureCollection to JSON format
state_geojson = geojson.dumps(state_feature_collection)
# check type to make sure conversion was sucessful
print(type(state_geojson))

<class 'str'>


In [35]:
# Save JSON-formatted FeatureCollection as JSON file
# Save as new json file
with open('JSON/state_geojson.json', 'w', encoding='utf-8') as json_file:
    json_file.write(state_geojson)

## Part 2 - OLD METHOD: Get state-level JSONs for all 3 databases

### 2b: Join each database dataframe to the state_centroids dataframe

In [ ]:
# check state centroids dataframe
state_centroids_df.head()

#### 1/4: Join state centroids dataframe to missing persons dataframe
##### (assign each row the coordinates of a state centroid based on the state fips code)

In [ ]:
# check dataframe
missing_df.head()

In [ ]:
# check dataframe length
len(missing_df)

In [ ]:
# join state centroids dataframe to missing persons dataframe - MAKE SURE LENGTH MATCHES LENGTH OF MISSING_DF
missing_states_df = pd.merge(left = missing_df, right = state_centroids_df, how = 'left', left_on = 'State_FIPS', right_on = 'STATEFP')
# len(missing_states_df)
missing_states_df.head()

#### 1/4: Join state centroids dataframe to unclaimed persons dataframe
##### (assign each row the coordinates of a state centroid based on the state fips code)

In [ ]:
# check dataframe
unclaimed_df.head()

In [ ]:
# check dataframe length
len(unclaimed_df)

In [ ]:
# join state centroids dataframe to unclaimed persons dataframe - MAKE SURE LENGTH MATCHES LENGTH OF UNCLAIMED_DF
unclaimed_states_df = pd.merge(left = unclaimed_df, right = state_centroids_df, how = 'left', left_on = 'State_FIPS', right_on = 'STATEFP')
# len(unclaimed_states_df)
unclaimed_states_df.head()

In [ ]:
# make sure int STATEFP joined ok to float State_FIPS [ NOTE: it joined fine]
# unclaimed_states_df[['State_FIPS', 'STATEFP']]

In [ ]:
## Double check rows w/ no FIPS to make sure missing_df data retained [NOTE: it was!]
unc_states_test_df = unclaimed_states_df.loc[unclaimed_states_df['State_FIPS'].isnull()]
unc_states_test_df

#### 1/4: Join state centroids dataframe to unidentified persons dataframe
##### (assign each row the coordinates of a state centroid based on the state fips code)

In [ ]:
# check dataframe
unidentified_df.head()

In [ ]:
# check dataframe length
len(unidentified_df)

In [ ]:
# join state centroids dataframe to unidentified persons dataframe - MAKE SURE LENGTH MATCHES LENGTH OF UNIDENTIFIED_DF
unidentified_states_df = pd.merge(left = unidentified_df, right = state_centroids_df, how = 'left', left_on = 'State_FIPS', right_on = 'STATEFP')
# len(unidentified_states_df)
unidentified_states_df.head()

#### 4/4: Convert dataframes to geodataframes and export as GeoJSON files

##### 4a: missing_states_df

In [ ]:
# check dataframe
missing_states_df.head()

In [ ]:
# Convert missing persons dataframe to geodataframe
missing_states_gdf = gpd.GeoDataFrame(missing_states_df, geometry=gpd.points_from_xy(x=missing_states_df.Lon_dd, y=missing_states_df.Lat_dd))
# len(missing_states_gdf)
missing_states_gdf.head()

In [ ]:
# write to geoJSON
missing_states_gdf.to_file("JSON/missing_states.json", driver="GeoJSON", encoding='utf-8')

##### 4b: unclaimed_states_df

In [ ]:
# check dataframe
unclaimed_states_df.head()

In [ ]:
# Convert unclaimed persons dataframe to geodataframe
unclaimed_states_gdf = gpd.GeoDataFrame(unclaimed_states_df, geometry=gpd.points_from_xy(x=unclaimed_states_df.Lon_dd, y=unclaimed_states_df.Lat_dd))
# len(unclaimed_states_gdf)
unclaimed_states_gdf.head()

In [ ]:
# write to geoJSON
unclaimed_states_gdf.to_file("JSON/unclaimed_states.json", driver="GeoJSON", encoding='utf-8')

##### 4c: unidentified_states_df

In [ ]:
# check dataframe
unidentified_states_df.head()

In [ ]:
# Convert unidentified persons dataframe to geodataframe
unidentified_states_gdf = gpd.GeoDataFrame(unidentified_states_df, geometry=gpd.points_from_xy(x=unidentified_states_df.Lon_dd, y=unidentified_states_df.Lat_dd))
# len(unidentified_states_gdf)
unidentified_states_gdf.head()

In [ ]:
# write to geoJSON
unidentified_states_gdf.to_file("JSON/unidentified_states.json", driver="GeoJSON", encoding='utf-8')

#### Check that GeoJSON files load properly

In [ ]:
test_miss_df = gpd.read_file('JSON/missing_states.json')
len(test_miss_df)
# test_miss_df.head()

In [ ]:
test_unc_df = gpd.read_file('JSON/unclaimed_states.json')
len(test_unc_df)
# test_unc_df.head()

In [ ]:
test_uni_df = gpd.read_file('JSON/unidentified_states.json')
len(test_uni_df)
# test_uni_df.head()

# Part 3: Get county-level JSONs for all 3 databases

# ADD IN STATE NAME TO COUNTY JSON

#### Add in County FIPS column to county centroids dataframe

In [98]:
# check county dataframe
# county_centroids_df.head()
len(county_centroids_df)

3227

In [99]:
# county_subset = county_centroids_df.loc[county_centroids_df['STATEFP']==35]
# county_subset

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
158,159,35,7,929117,0500000US35007,35007,Colfax,6,9.733110e+09,26299632,159,-104.646850,36.606145
159,160,35,11,933054,0500000US35011,35011,De Baca,6,6.016819e+09,29089486,160,-104.412033,34.342458
160,161,35,51,1702370,0500000US35051,35051,Sierra,6,1.082925e+10,147676270,161,-107.192417,33.130501
546,547,35,15,936829,0500000US35015,35015,Eddy,6,1.081690e+10,56282426,547,-104.304317,32.471491
547,548,35,39,1702368,0500000US35039,35039,Rio Arriba,6,1.517959e+10,91468728,548,-106.693153,36.509583
548,549,35,45,936844,0500000US35045,35045,San Juan,6,1.428949e+10,54969313,549,-108.320632,36.508530
549,550,35,55,933056,0500000US35055,35055,Taos,6,5.704119e+09,3383128,550,-105.630972,36.578308
891,892,35,19,929111,0500000US35019,35019,Guadalupe,6,7.847072e+09,2887948,892,-104.790666,34.863311
915,916,35,33,1702367,0500000US35033,35033,Mora,6,4.988963e+09,6093304,916,-104.944827,36.010404
987,988,35,17,915980,0500000US35017,35017,Grant,6,1.025943e+10,15345466,988,-108.382413,32.738879


In [100]:
# Add column with compound field key
county_centroids_df['County_Key'] = county_centroids_df['STATEFP'].astype(str) + "_" + county_centroids_df['NAME']
county_centroids_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd,County_Key
0,1,21,7,516850,0500000US21007,21007,Ballard,6,6.393875e+08,69473325,1,-88.999262,37.058489,21_Ballard
1,2,21,17,516855,0500000US21017,21017,Bourbon,6,7.504394e+08,4829777,2,-84.217155,38.206742,21_Bourbon
2,3,21,31,516862,0500000US21031,21031,Butler,6,1.103572e+09,13943044,3,-86.681628,37.207292,21_Butler
3,4,21,65,516879,0500000US21065,21065,Estill,6,6.555099e+08,6516335,4,-83.964316,37.692451,21_Estill
4,5,21,69,516881,0500000US21069,21069,Fleming,6,9.027272e+08,7182793,5,-83.696660,38.370126,21_Fleming


In [101]:
# check unique values and length
county_key_list = county_centroids_df['County_Key'].unique()
len(county_key_list)

3227

In [102]:
# make dictionary of counties and county FIPS code (GEIOD field)
county_dict = dict(zip(county_centroids_df.County_Key, county_centroids_df.GEOID))
county_dict

{'21_Ballard': 21007,
 '21_Bourbon': 21017,
 '21_Butler': 21031,
 '21_Estill': 21065,
 '21_Fleming': 21069,
 '21_Hardin': 21093,
 '21_Hart': 21099,
 '21_Leslie': 21131,
 '21_Madison': 21151,
 '21_Marion': 21155,
 '21_Nelson': 21179,
 '17_Kankakee': 17091,
 '17_Warren': 17187,
 '17_Will': 17197,
 '18_Daviess': 18027,
 '18_Harrison': 18061,
 '1_Autauga': 1001,
 '1_Barbour': 1005,
 '1_Choctaw': 1023,
 '1_Conecuh': 1035,
 '1_Elmore': 1051,
 '1_Etowah': 1055,
 '1_Hale': 1065,
 '1_Madison': 1089,
 '1_Russell': 1113,
 '1_Shelby': 1117,
 '2_Aleutians West': 2016,
 '2_Ketchikan Gateway': 2130,
 '2_Nome': 2180,
 '2_Yakutat': 2282,
 '5_Bradley': 5011,
 '5_Carroll': 5015,
 '5_Clark': 5019,
 '5_Cleveland': 5025,
 '5_Independence': 5063,
 '5_Jackson': 5067,
 '5_Randolph': 5121,
 '6_Alameda': 6001,
 '6_Alpine': 6003,
 '6_Amador': 6005,
 '6_Contra Costa': 6013,
 '6_Fresno': 6019,
 '6_Humboldt': 6023,
 '6_Lake': 6033,
 '6_Los Angeles': 6037,
 '6_Merced': 6047,
 '6_Nevada': 6057,
 '6_San Mateo': 6081,
 

#### Add in County FIPS column to missing dataframe

In [103]:
# check missing df
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


In [104]:
# Add column with compound field key
missing_df['State_County'] = missing_df['State_FIPS'].astype(str) + "_" + missing_df['County']
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin


In [105]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
missing_df['County_FIPS'] = missing_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
missing_df['County_FIPS'].unique()

array([ 1001.,  1003.,  1005., ..., 56035., 56037., 56039.])

In [112]:
# check null values [NOTE: all ok - no city or county assigned]
missing_county_null_df = missing_df.loc[missing_df['County_FIPS'].isnull()]
# missing_county_null_df.shape
missing_county_null_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna Aponte,John,12,NaN,NaN,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,NaN,NaN
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,NaN,Puerto Rico,Male,Black / African American,3/12/2020,72,NaN,NaN
14183,MP4568,5/4/1974,Colonna Aponte,Giannina,11,NaN,NaN,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,NaN,NaN


In [109]:
# As needed, export nulls to address
# missing_county_null_df.to_csv('missing_county_nulls.csv', encoding='Windows-1252')

In [118]:
# check missing_df
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga,1001.0
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga,1001.0
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin,1003.0
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin,1003.0
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin,1003.0


#### Add in County FIPS column to unclaimed dataframe

In [111]:
# check unclaimed df
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20,36.0
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20,36.0
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20,36.0


In [114]:
# Add column with compound field key
unclaimed_df['State_County'] = unclaimed_df['State_FIPS'].astype(str) + "_" + unclaimed_df['County']
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20,36.0,36.0_New York
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20,36.0,36.0_Kings
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20,36.0,36.0_New York


In [115]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unclaimed_df['County_FIPS'] = unclaimed_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
unclaimed_df['County_FIPS'].unique()

array([nan])

In [116]:
# check null values
unclaimed_county_null_df = unclaimed_df.loc[unclaimed_df['County_FIPS'].isnull()]
# unclaimed_county_null_df.shape
unclaimed_county_null_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will,NaN
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will,NaN
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20,36.0,36.0_New York,NaN
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20,36.0,36.0_Kings,NaN
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20,36.0,36.0_New York,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8330,UCP320,NaN,Baskin,Clyde,Male,Black / African American,Spokane Valley,Spokane,Washington,4/14/20,53.0,53.0_Spokane,NaN
8331,UCP341,NaN,Brotherton,Quilen,Male,NaN,NaN,Spokane,Washington,4/14/20,53.0,53.0_Spokane,NaN
8332,UCP326,NaN,Bell,Phillip,Male,NaN,NaN,Spokane,Washington,4/14/20,53.0,53.0_Spokane,NaN
8333,UCP345,NaN,Buonantony,Elena,Female,NaN,NaN,Spokane,Washington,4/14/20,53.0,53.0_Spokane,NaN


In [109]:
# As needed, export nulls to address
# unclaimed_county_null_df.to_csv('unclaimed_county_nulls.csv', encoding='Windows-1252')

In [117]:
# check unclaimed_df
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/20,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will,NaN
1,UCP68248,4/13/20,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/20,17.0,17.0_Will,NaN
2,UCP68242,4/9/20,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/20,36.0,36.0_New York,NaN
3,UCP68244,4/9/20,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/20,36.0,36.0_Kings,NaN
4,UCP68228,4/9/20,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/20,36.0,36.0_New York,NaN


#### Add in County FIPS column to unidentified dataframe

In [119]:
# check unidentified df
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020,36
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020,41
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020,36
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020,36
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020,40


In [120]:
# Add column with compound field key
unidentified_df['State_County'] = unidentified_df['State_FIPS'].astype(str) + "_" + unidentified_df['County']
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020,36,36_Queens
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020,41,41_Linn
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020,36,36_Queens
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020,36,36_New York
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020,40,40_Cleveland


In [121]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unidentified_df['County_FIPS'] = unidentified_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
unidentified_df['County_FIPS'].unique()

array([36081., 41043., 36061., ...,  8047., 54065., 50001.])

In [122]:
# check null values
unidentified_county_null_df = unidentified_df.loc[unidentified_df['County_FIPS'].isnull()]
# unidentified_county_null_df.shape
unidentified_county_null_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
51,UP67784,1/12/2020,NaN,NaN,SAN JUAN,All,Puerto Rico,Male,Hispanic / Latino,4/2/2020,72,72_All,NaN
55,UP67337,1/9/2020,NaN,NaN,Vega Baja,All,Puerto Rico,Male,Hispanic / Latino,4/2/2020,72,72_All,NaN
57,UP65131,1/4/2020,NaN,NaN,SAN JUAN,All,Puerto Rico,Male,Hispanic / Latino,4/2/2020,72,72_All,NaN
58,UP64754,1/3/2020,NaN,NaN,SAN JUAN,All,Puerto Rico,Male,Hispanic / Latino,4/2/2020,72,72_All,NaN
60,UP65129,12/31/2019,NaN,NaN,CULEBRA,All,Puerto Rico,Male,Hispanic / Latino,4/2/2020,72,72_All,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12572,UP11056,5/14/1985,18.0,59.0,NaN,All,Guam,Unsure,Uncertain,8/15/2018,66,66_All,NaN
12584,UP8314,3/10/1985,16.0,20.0,Dona Ana,NaN,New Mexico,Female,White / Caucasian,2/8/2020,35,NaN,NaN
12638,UP12846,4/25/1984,19.0,20.0,Palmer,NaN,Alaska,Female,White / Caucasian,2/13/2020,2,NaN,NaN
12974,UP4435,12/1/1977,30.0,50.0,Odessa,NaN,Texas,Unsure,Uncertain,7/30/2019,48,NaN,NaN


In [128]:
# As needed, export nulls to address
unidentified_county_null_df.to_csv('unidentified_county_nulls.csv', encoding='Windows-1252')

In [127]:
# check unclaimed_df
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,UP68289,4/15/2020,35.0,60.0,Queens,Queens,New York,Male,Hispanic / Latino,4/18/2020,36,36_Queens,36081.0
1,UP68210,4/10/2020,40.0,60.0,NaN,Linn,Oregon,Male,White / Caucasian,4/15/2020,41,41_Linn,41043.0
2,UP68081,4/10/2020,30.0,40.0,Queens,Queens,New York,Male,Hispanic / Latino,4/12/2020,36,36_Queens,36081.0
3,UP68075,4/10/2020,25.0,50.0,New York,New York,New York,Male,White / Caucasian,4/12/2020,36,36_New York,36061.0
4,UP68204,4/6/2020,NaN,NaN,Oklahoma City,Cleveland,Oklahoma,Male,White / Caucasian,4/16/2020,40,40_Cleveland,40027.0
